# Fine-tuning a Mental Health Support Chatbot Using DistilGPT-2

This notebook demonstrates the process of fine-tuning a lightweight GPT-2 model (`distilgpt2`) to function as an empathetic mental health support chatbot. A synthetic dataset of user concerns paired with supportive assistant responses is created to train the model.

### Workflow Overview:
- Install and import required libraries
- Load a pre-trained transformer model
- Construct a synthetic mental health conversation dataset
- Tokenize and preprocess the dataset
- Fine-tune the model using Hugging Face's `Trainer` API
- Perform inference to generate supportive chatbot responses

The goal of this notebook is to provide an educational example of adapting a general-purpose language model into a task-specific conversational assistant through minimal fine-tuning.

### Installing Required Libraries
This cell installs the `transformers` and `datasets` libraries from Hugging Face, which are essential for building and training language models.


In [1]:
# Install necessary packages silently
!pip install -q transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==1

### Importing Libraries
This cell imports PyTorch, Hugging Face datasets, and various modules from the `transformers` library required for tokenization, model loading, training setup, and data collation.


In [3]:
# Import essential libraries
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)


### Loading Pre-trained Model
This cell loads a lightweight GPT-2 variant (`distilgpt2`) and prepares the tokenizer and model, making sure the padding token is properly set.


In [4]:
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### Creating a Rich Synthetic Dataset
This cell defines a broader dataset simulating various user struggles around anxiety, loneliness, low self-esteem, and other mental health concerns, each paired with an empathetic assistant response.


In [5]:
data = {
    "text": [
        "User: I'm feeling really anxious lately.\nAssistant: I'm really sorry you're feeling this way. Remember, anxiety can be managed step by step. You're not alone, and I'm here for you.",
        "User: I don't know how to deal with stress at work.\nAssistant: It's understandable to feel overwhelmed. Small breaks and breathing exercises during the day can help ground you.",
        "User: I feel like nobody understands me.\nAssistant: That must be tough. Please remember that your feelings are valid, and connecting with supportive people can help you feel less isolated.",
        "User: I can't sleep because of constant worries.\nAssistant: Sleep can be hard with a racing mind. Writing down your worries or listening to calming music might help relax you.",
        "User: I feel hopeless about the future.\nAssistant: It's okay to feel lost sometimes. Try to focus on small, positive steps you can control each day. Hope can return slowly.",
        "User: I'm afraid of failing at everything.\nAssistant: Fear of failure is natural, but it doesn't define your worth. Every small effort you make is a step forward.",
        "User: I have no motivation to get out of bed.\nAssistant: Lack of motivation can be a sign of burnout or depression. Be gentle with yourself and celebrate even small achievements.",
        "User: I'm scared of talking to new people.\nAssistant: Social anxiety can be very real. Starting with small conversations and practicing self-kindness can make a big difference over time.",
        "User: I can't stop overthinking everything.\nAssistant: Overthinking can drain your energy. Sometimes setting time limits on thinking and journaling thoughts can provide relief.",
        "User: I'm feeling very lonely.\nAssistant: Loneliness can be really painful. Reaching out, even digitally, or joining communities around your interests can help you feel connected.",
        "User: My mind feels constantly restless.\nAssistant: A restless mind can be exhausting. Gentle activities like mindfulness, walks, or slow breathing can bring a sense of peace.",
        "User: I'm always doubting myself.\nAssistant: Self-doubt is something everyone faces. Try to be as kind to yourself as you would to a good friend in your situation.",
        "User: I feel like I'm stuck in life.\nAssistant: Feeling stuck is hard. Sometimes small changes in routine, seeking new experiences, or talking it through with someone can open new paths.",
        "User: I'm panicking over small things.\nAssistant: It's okay to feel overwhelmed. Focusing on grounding techniques, like naming five things you see around you, can help calm the mind.",
        "User: I don't think I'm good enough.\nAssistant: You are more capable than you realize. Try to challenge those negative thoughts by focusing on your strengths and achievements."
    ]
}


### Wrapping Data into Dataset
This cell wraps the predefined conversation data into a `Dataset` object, allowing easier preprocessing and model training.


In [6]:
# Convert dictionary data into a Hugging Face Dataset object
dataset = Dataset.from_dict(data)

### Tokenizing Dataset
This cell tokenizes the text data, ensuring that each example is properly padded and truncated, so it can be fed into the model during training.


In [7]:
# Tokenize dataset entries
def tokenize_example(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Apply tokenization across the dataset
tokenized_dataset = dataset.map(tokenize_example)


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

### Setting Up Trainer
This cell configures the training parameters and initializes a Hugging Face `Trainer`, which manages the model training process automatically.


In [8]:
# ✅ Define training configurations
training_args = TrainingArguments(
    output_dir="./mental-health-chatbot",
    per_device_train_batch_size=1,
    num_train_epochs=2,
    logging_steps=1,
    save_total_limit=1,
    report_to="none"  # Disable logging to external services
)

# ✅ Setup the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # Use causal language modeling
    )
)


<ipython-input-8-7f8d1bd7c9ac>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
# ✅ Start model fine-tuning
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,4.531500
2,4.504400
3,4.503100
4,4.023400
5,3.442700
6,3.434000
7,3.836800
8,3.247000
9,3.472100
10,2.954700


TrainOutput(global_step=30, training_loss=3.1782338619232178, metrics={'train_runtime': 7.7898, 'train_samples_per_second': 3.851, 'train_steps_per_second': 3.851, 'total_flos': 979862814720.0, 'train_loss': 3.1782338619232178, 'epoch': 2.0})

### Inference with the Fine-tuned Bot
This cell sends a sample prompt to the fine-tuned chatbot and generates a supportive response, demonstrating the model's empathetic behavior after training.


In [11]:
# Define a prompt for the chatbot
prompt = "User: I'm feeling very stressed at work.\nAssistant:"
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True
)

# Move inputs to same device as model
device = model.device
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate assistant response
output = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=50,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)

# Decode and print the result
print(tokenizer.decode(output[0], skip_special_tokens=True))


User: I'm feeling very stressed at work.
Assistant: Work is your best chance. You can't stop feeling stressed.
Assistant: Take a deep breath. Work with your mind to let yourself feel stressed.
Assistant: Take a deep breath. Breathe and talk to yourself.
Assistant: Breat
